In [ ]:
# Für Google Colab: Upload & Download
from google.colab import files
import os
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_CHART_TYPE
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR

# Datei-Upload
uploaded = files.upload()
input_pptx = list(uploaded.keys())[0]
output_pptx = f"bearbeitet_{input_pptx}"

def add_bullet_textbox(prs, slide, text_list, font_name="Frutiger 44 Light", font_size=Pt(16), font_color=RGBColor(0, 0, 0)):
    left = Inches(7.5)
    top = Inches(1.5)
    width = Inches(5)
    height = Inches(5)
    textbox = slide.shapes.add_textbox(left, top, width, height)
    text_frame = textbox.text_frame
    text_frame.word_wrap = True
    text_frame.vertical_anchor = MSO_ANCHOR.TOP
    for i, text in enumerate(text_list):
        p = text_frame.paragraphs[0] if i == 0 else text_frame.add_paragraph()
        p.text = f"• {text}"
        p.font.size = font_size
        p.font.name = font_name
        p.font.color.rgb = font_color
        p.font.bold = True
        p.alignment = PP_ALIGN.LEFT

def set_background_color(slide, color):
    fill = slide.background.fill
    fill.solid()
    fill.fore_color.rgb = color

def color_bottom_slide(prs, slide, color):
    slide_height = prs.slide_height
    height = slide_height // 20
    shape = slide.shapes.add_shape(1, Inches(0), slide_height - height, prs.slide_width, height)
    shape.fill.solid()
    shape.fill.fore_color.rgb = color
    shape.line.color.rgb = color

def style_textbox(slide, color):
    text_boxes = [s for s in slide.shapes if s.has_text_frame]
    if len(text_boxes) > 0:
        text_boxes[0].text_frame.clear()
        paragraph = text_boxes[0].text_frame.add_paragraph()
        run_title = paragraph.add_run()
        run_title.text = 'TITEL EINFÜGEN'
        run_title.font.size = Pt(20)
        run_title.font.bold = True
        run_title.font.name = "Frutiger 44 Light"
    if len(text_boxes) > 1:
        tb = text_boxes[1]
        tb.line.color.rgb = color
        tb.line.width = Pt(1)
        tb.height = tb.height // 2
        existing_text = tb.text_frame.text.strip()
        tb.text_frame.clear()
        p = tb.text_frame.paragraphs[0] if tb.text_frame.paragraphs else tb.text_frame.add_paragraph()
        run_bold = p.add_run()
        run_bold.text = "Frage: "
        run_bold.font.bold = True
        run_bold.font.size = Pt(10)
        run_bold.font.name = "Frutiger 44 Light"
        run_normal = p.add_run()
        run_normal.text = existing_text
        run_normal.font.size = Pt(10)
        run_normal.font.name = "Frutiger 44 Light"

def modify_bar_chart_colors(chart, colors):
    for i, series in enumerate(chart.plots[0].series):
        series.format.fill.solid()
        series.format.fill.fore_color.rgb = colors[i % len(colors)]
        for point in series.points:
            point.format.fill.solid()
            point.format.fill.fore_color.rgb = colors[i % len(colors)]

def modify_pie_chart_colors(chart, colors):
    for i, point in enumerate(chart.plots[0].series[0].points):
        point.format.fill.solid()
        point.format.fill.fore_color.rgb = colors[i % len(colors)]

def modify_bar_chart(chart, shape, prs, font_color, colors):
    modify_bar_chart_colors(chart, colors)
    chart.value_axis.minimum_scale = 0
    chart.value_axis.maximum_scale = 1
    shape.top = int(prs.slide_height / 5)
    shape.width = int(prs.slide_width / 2)

def modify_pie_chart(chart, shape, prs, font_color, colors):
    modify_pie_chart_colors(chart, colors)
    shape.left = 0
    shape.top = int(prs.slide_height / 5)
    shape.width = int(prs.slide_width / 2)

def modify_presentation(input_pptx, output_pptx):
    prs = Presentation(input_pptx)
    background_color = RGBColor(174, 177, 192)
    bottom_color = RGBColor(255, 105, 86)
    font_color = RGBColor(0, 0, 0)
    chart_colors = [
        RGBColor(255, 105, 86),
        RGBColor(232, 230, 215),
        RGBColor(255, 255, 255),
        RGBColor(0, 0, 0)
    ]

    for slide in prs.slides:
        set_background_color(slide, background_color)
        color_bottom_slide(prs, slide, bottom_color)
        style_textbox(slide, RGBColor(255, 105, 86))
        add_bullet_textbox(prs, slide, ["Erster Punkt", "Zweiter Punkt"])

        for shape in slide.shapes:
            if shape.has_chart:
                chart = shape.chart
                chart_type = chart.chart_type
                if chart_type in [
                    XL_CHART_TYPE.COLUMN_CLUSTERED,
                    XL_CHART_TYPE.COLUMN_STACKED,
                    XL_CHART_TYPE.COLUMN_STACKED_100,
                    XL_CHART_TYPE.BAR_CLUSTERED,
                    XL_CHART_TYPE.BAR_STACKED,
                    XL_CHART_TYPE.BAR_STACKED_100
                ]:
                    modify_bar_chart(chart, shape, prs, font_color, chart_colors)
                elif chart_type == XL_CHART_TYPE.PIE:
                    modify_pie_chart(chart, shape, prs, font_color, chart_colors)

    prs.save(output_pptx)

# Ausführen
modify_presentation(input_pptx, output_pptx)

# Download-Link bereitstellen
files.download(output_pptx)
